In [2]:
import os
import pandas as pd
import numpy as np
import pylidc as pl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import math
import pandas as pd

In [43]:
df_pl = pd.DataFrame(columns= [
                            'patient_id', 
                            'annotation_id',
                            'scan_id',

                            'subtlety', 
                            'internalStructure', 
                            'calcification', 
                            'sphericity', 
                            'margin', 
                            'lobulation', 
                            'spiculation', 
                            'texture', 
                        
                            'diameter',
                            'surface_area',
                            'volume',

                            'malignancy',
                            ])



In [44]:
ann = pl.query(pl.Annotation).all()

for i in range(len(ann)):
    att  = dict((col, "") for col in df_pl.columns)

    # patient and annotation identification
    att['patient_id'] = ann[i].scan.patient_id
    att['annotation_id'] = ann[i].id  
    att['scan_id'] = ann[i].scan.id

    # features
    #st = pl.query(pl.Scan.slice_thickness).filter(pl.Scan.id == att['scan_id'])
    #s = str(st[0])
    #att['slice_thickness'] = float(s[1:4])
    #ps = pl.query(pl.Scan.pixel_spacing).filter(pl.Scan.id == att['scan_id'])
    #p = str(ps[0])
    #att['pixel_spacing'] = float(p[1:5])

    att['subtlety'] = ann[i].subtlety
    att['internalStructure'] = ann[i].internalStructure 
    att['calcification'] = ann[i].calcification 
    att['sphericity'] = ann[i].sphericity
    att['margin'] = ann[i].margin  
    att['lobulation'] = ann[i].lobulation
    att['spiculation'] = ann[i].spiculation 
    att['texture'] = ann[i].texture

    att['diameter'] = ann[i].diameter
    att['surface_area'] = ann[i].surface_area
    att['volume'] = ann[i].volume

    # target
    att['malignancy'] = ann[i].malignancy  

    df_pl = df_pl.append(att, ignore_index=True)


In [45]:
df_pl.sort_values(by=['patient_id', 'annotation_id'], inplace=True)

df_pl['Ann_id'] = df_pl.groupby('patient_id').cumcount() + 1
df_pl = df_pl[['Ann_id'] + [col for col in df_pl.columns if col != 'Ann_id']]

################################################################################################

for i in range(len(df_pl)):
    df_pl.at[i,'Id'] = df_pl.at[i,'patient_id'] + '-' + str(df_pl.at[i,'Ann_id'])

df_pl = df_pl[['Id'] + [col for col in df_pl.columns if col != 'Id']]
df_pl = df_pl.drop(columns=['Ann_id', 'annotation_id', 'scan_id'])

df_pl.to_csv('all_features.csv', sep=',', index=False)

In [46]:
# Pasta raiz onde as pastas estão localizadas
pasta_raiz = "LIDC-IDRI"

# Extrair números das pastas que você possui
pastas_existente = [folder.split("-")[-1] for folder in os.listdir(pasta_raiz) if folder.startswith("LIDC-IDRI-")]

# Ler o arquivo CSV em um DataFrame Pandas
df = pd.read_csv("all_features.csv")

# Criar uma coluna no DataFrame com os números das pastas
df['numero_pasta'] = df['patient_id'].str.split('-').str[-1]

# Filtrar o DataFrame para manter apenas as entradas correspondentes às pastas existentes
df_filtrado = df[df['numero_pasta'].isin(pastas_existente)]

df_filtrado.to_csv('our.csv', sep=',', index=False)

#funciona yey

# Pyradiomics

In [47]:

main_directory = 'LIDC-IDRI'

for patient in os.listdir(main_directory):
    patient_directory = os.path.join(main_directory, patient)

    if not os.path.isdir(patient_directory):
        continue

    for root, dirs, files in os.walk(patient_directory):
        for file in files:
            if file.endswith('.dcm'):
                dcm_file_path = os.path.join(root, file)
                print(f'Found .dcm file: {dcm_file_path}')

                # Se você quiser usar esse caminho no seu script "pyradiomics-dcm.py",
                # você pode adicioná-lo aos argumentos, conforme necessário.
                # Exemplo:
                script_args = [
                    'python',
                    'pyradiomics-dcm.py',
                    f'--input-image-file="{dcm_file_path}"',
                    '--output-dir=OutputSR',
                    '--temp-dir=TempDir',
                    '--parameters=Pyradiomics_Params.yaml',
                    '--features-dict=featuresDict.tsv',
                    f'--name={patient}-{seg_index}'
                ]
                os.system(' '.join(script_args))


Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CHEST PA - LAT-67107\22137.000000-CHEST PA - LAT-12560\1-1.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CT CHEST W CONT-80168\4.000000-Recon 3 CT CHEST-26125\1-001.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CT CHEST W CONT-80168\4.000000-Recon 3 CT CHEST-26125\1-002.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CT CHEST W CONT-80168\4.000000-Recon 3 CT CHEST-26125\1-003.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CT CHEST W CONT-80168\4.000000-Recon 3 CT CHEST-26125\1-004.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CT CHEST W CONT-80168\4.000000-Recon 3 CT CHEST-26125\1-005.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CT CHEST W CONT-80168\4.000000-Recon 3 CT CHEST-26125\1-006.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068\01-01-2000-NA-CT CHEST W CONT-80168\4.000000-Recon 3 CT CHEST-26125\1-007.dcm
Found .dcm file: LIDC-IDRI\LIDC-IDRI-0068

KeyboardInterrupt: 